In [ ]:
from prodock.io.pdb_query import PDBQuery
proc = PDBQuery("5N2F", "Data/testcase", chains=["A", "B"], ligand_code="8HW")
proc.run_all()
print("raw:", proc.cocrystal_ligand_path)
print("processed:", proc.reference_ligand_path)

In [ ]:
from prodock.preprocess.gridbox import GridBox
from prodock.vis.provis import ProVis
from prodock.vis.provis_gui import ProVisGUI


In [ ]:
gb = GridBox().load_ligand("Data/testcase/cocrystal/5N2F.sdf", fmt="sdf") \
              .from_ligand_pad(pad=4.0, isotropic=True)

print("Box dict for Vina:", gb.vina_dict)
print("Box config snippet:\n", gb.to_vina_lines())


In [ ]:
viz = ProVis(800, 600)
viz.load_receptor("Data/testcase/filtered_protein/5N2F.pdb") \
   .set_receptor_style("cartoon", "white") \
   .load_ligand("Data/testcase/reference_ligand/8HW.sdf", fmt="sdf") \
   .highlight_ligand(style="stick", color="cyan") \
   .add_gridbox_from(gb, labels=True) \
   .show()


In [ ]:
gui = ProVisGUI().build().display()


In [ ]:
from prodock.preprocess.protein_process import ProteinProcess

pp = ProteinProcess(enable_logging=True)
pp.fix_and_minimize_pdb(
    input_pdb="Data/testcase/filtered_protein/5N2F.pdb",
    output_dir="Data/testcase/filtered_protein/",
    energy_diff=10.0,
    max_minimization_steps=5000,
    pdb_id="5N2F",
    protein_name="5N2F",
    minimize_in_water=False,
    input_ligand='Data/testcase/reference_ligand/8HW.sdf',
    out_fmt="gpf",   # default; will produce Data/testcase/filtered_protein/5N2F_min.pdb
)

# print("Artifact:", pp.final_pdb_path)
print("Report:", pp.last_simulation_report)


In [ ]:
from prodock.io.convert import Converter
conv = Converter().set_input("Data/testcase/reference_ligand/8HW.sdf").set_output("Data/testcase/reference_ligand/8HW.pdbqt").set_mode("ligand").use_meeko().run()
print("output:", conv.output)
print("requested mode:", conv._mode)        # 'ligand'
print("preferred backend:", conv._prefer)  # 'meeko' (because you called use_meeko())


In [ ]:
from prodock.engine.vina import VinaDock

In [ ]:
vd = (VinaDock(sf_name="vina", cpu=4, seed=42)
      .set_receptor("Data/testcase/filtered_protein/5N2F.pdbqt")
      .define_box(center=(32.5, 13.0, 133.75), size=(22.5, 23.5, 22.5))
)

# single ligand
vd.set_ligand("Data/testcase/reference_ligand/8HW.pdbqt").dock(exhaustiveness=8, n_poses=9).write_poses("./Data/testcase/vina/vina_out.pdbqt").write_log("./Data/testcase/vina/log.txt")
print("scores:", vd.scores)
print("best:", vd.get_best())


In [ ]:
from prodock.engine.binary import BinaryDock

In [ ]:
bd = (
    BinaryDock("qvina-w")
    .set_receptor("5N2F.pdbqt")
    .set_ligand("8HW.pdbqt")
    .set_out("docked/8HW_docked.pdbqt")
    .set_log("docked/8HW.log")
    .set_box( center=(32.500, 13.0, 133.750), size=(22.500, 23.500, 22.500))
    # .enable_autobox("Data/testcase/reference_ligand/8HW.sdf", padding=4.0)
    .set_exhaustiveness(2)
    .set_num_modes(9)
    .set_cpu(4)
    .set_seed(42)
    .run()
)
# parse rows
rows = bd.parse_scores_from_log("docked/8HW.log")
print(rows)
# write CSV
bd.scores_to_csv("docked/8HW.log", "docked/8HW_scores.csv")
# get pandas DataFrame (requires pandas)
df = bd.scores_as_dataframe("docked/8HW.log")
print(df)


In [ ]:
from prodock.io.rdkit_io import smiles2sdf
smiles2sdf("c1ccccc1", "test.sdf", embed3d=True, add_hs=True, optimize=True)

In [1]:
from prodock.process.ligand import LigandProcess 

rows = [{"smiles": "CCO", "name": "ethanol"},
       {"smiles": "c1ccccc1", "name": "benzene"}]

lp = LigandProcess(output_dir='out', name_key="name")
lp.from_list_of_dicts(rows)
lp.set_embed_method("ETKDGv3").set_opt_method("MMFF94s")
lp.process_all()

2025-08-27 15:10:42 | INFO | rdkit | Enabling RDKit 2025.03.5 jupyter extensions
2025-08-27 15:10:42 | INFO | prodock.chem.embed | Embedder: loaded 1 SMILES from iterable
2025-08-27 15:10:42 | INFO | prodock.chem.embed | Embedder: finished embedding: 1 successes / 1 attempts
2025-08-27 15:10:42 | INFO | prodock.chem.conformer | ConformerManager: embedded 1 / 1 molecules
2025-08-27 15:10:42 | INFO | prodock.chem.optimize | Optimizer: loaded 1 MolBlocks for optimization
2025-08-27 15:10:42 | INFO | prodock.chem.optimize | Optimizer: finished optimization: 1 succeeded
2025-08-27 15:10:42 | INFO | prodock.chem.conformer | ConformerManager: optimized 1 molecules
2025-08-27 15:10:42 | INFO | prodock.ligand.process | Processed record 0 (ethanol) -> out/ethanol.sdf
2025-08-27 15:10:42 | INFO | prodock.chem.embed | Embedder: loaded 1 SMILES from iterable
2025-08-27 15:10:42 | INFO | prodock.chem.embed | Embedder: finished embedding: 1 successes / 1 attempts
2025-08-27 15:10:42 | INFO | prodock.

<LigandProcess: 2 entries, ok=2, failed=0>